In [ ]:
!ls /content/drive/My\ Drive/ML_Project2_2022/tmp_bert_models/ML-project2/src_bert

build_model.py	plots.ipynb    run.ipynb	train.ipynb
dataloader.py	preprocess.py  run.py		train.py
helpers.py	__pycache__    _terminal.ipynb


In [ ]:
# on shared drive
from google.colab import drive
drive.mount('/content/drive')

# change to work directory
%cd '/content/drive/My Drive/ML_Project2_2022/tmp_bert_models/ML-project2/src_bert'
!ls ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1_UeL7yXCBK5FuVDhV-FhEJCOyuYyf6AC/ML-project2/src_bert
build_model.py	plots.ipynb    run.ipynb	train.ipynb
dataloader.py	preprocess.py  run.py		train.py
helpers.py	__pycache__    _terminal.ipynb


- - -

`ipynb` version of *train.py*

## 0 - Environment setting for Google Colab

In [ ]:
# if run this notebook in Google Colab
from google.colab import drive
drive.mount('/content/drive')

# change to work directory
%cd '/content/drive/My Drive/Colab Notebooks/ML-project2/src_bert'
!ls ./

Mounted at /content/drive
[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/ML-project2/src_bert'
/content
drive  sample_data


In [ ]:
# if run this notebook in Google Colab
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 99.8 MB/s 
     |████████████████████████████████| 7.6 MB 84.3 MB/s 


## Imports and Pamareters

In [ ]:
train_path = "../data/split/partial/train.tsv"
test_path = "../data/split/partial/test.tsv"
model_path = "../models/bert_model.h5"      # path to save model
result_path = "../results/tmp.csv"        # path to save convergence curve

In [ ]:
import csv
import numpy as np
import transformers as ppb
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from build_model import DNN, BERTModel, build_model
from dataloader import DatasetBERT

TENSOR_DTYPE = torch.float32
torch.set_default_dtype(TENSOR_DTYPE)
# check if GPU available
print("Pytorch version:", torch.__version__, end=" | ")
print("gpu available:", torch.cuda.is_available(), ",", torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Pytorch version: 1.13.0+cu116 | gpu available: True , A100-SXM4-40GB
1


## Build Model

In [ ]:
# Load pre-trained BERT base
bert_base = ppb.DistilBertModel.from_pretrained('distilbert-base-uncased').cuda()
# bert_base = ppb.RobertaModel.from_pretrained('roberta-base').cuda()

top_clf = DNN(input_size=768, hidden_size=128)
bert_model = BERTModel(bert_base, top_clf)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Load Data

In [ ]:
dataset_tr = DatasetBERT(train_path)
dataset_te = DatasetBERT(test_path)

## Train Model

### Train Top-classifier

In [ ]:
np.random.seed(114514)

# Parameters
lr = 1e-5
batch_size = 128 #64
maxiter = 10000
validation_per = 100
optimizer_train = optim.RMSprop(top_clf.parameters(), lr=lr)
scheduler = None #ExponentialLR(optimizer_train, gamma=0.999)

# Dataloader
train_loader = DataLoader(dataset_tr, batch_size=batch_size)#, num_workers=2)
valid_loader = DataLoader(dataset_te, batch_size=batch_size)#, num_workers=2)

# Start training
print("\n---------- Train top-classifier ----------\n")
results_train = build_model(bert_model, 
    lr = lr,
    train_loader = train_loader,
    test_loader = valid_loader,
    criterion = F.mse_loss,
    optimizer = optimizer_train,
    max_iter = maxiter,
    validate_per = validation_per,
    scheduler = scheduler
)



---------- Train top-classifier ----------

iter:  100                 loss_tr:0.243768527507782 acc_tr:0.611875                 loss_te:0.23920850413785855 acc_te:0.6536199433702818
iter:  200                 loss_tr:0.23718171775341035 acc_tr:0.64046875                 loss_te:0.23347668454680645 acc_te:0.6644055515947477
iter:  300                 loss_tr:0.2319110155105591 acc_tr:0.65453125                 loss_te:0.228612192709681 acc_te:0.672051920017726
iter:  400                 loss_tr:0.22697037801146508 acc_tr:0.6684375                 loss_te:0.2240000495188673 acc_te:0.6764813762315562
iter:  500                 loss_tr:0.22352904558181763 acc_tr:0.661953125                 loss_te:0.2201477973393991 acc_te:0.6832755115670217
iter:  600                 loss_tr:0.22092122375965118 acc_tr:0.6721875                 loss_te:0.2163546083258911 acc_te:0.6831115375102406
iter:  700                 loss_tr:0.21649822622537612 acc_tr:0.681953125                 loss_te:0.212905124

In [ ]:
with open("../results/bert_train_1", "w") as f:
   writer = csv.writer(f)
   writer.writerow(results_train.keys())
   writer.writerows(zip(*results_train.values()))

In [ ]:
results_train = build_model(bert_model, 
    lr = lr,
    train_loader = train_loader,
    test_loader = valid_loader,
    criterion = F.mse_loss,
    optimizer = optimizer_train,
    max_iter = 2000,
    validate_per = validation_per,
    scheduler = scheduler
)
with open("../results/bert_train_3.csv", "w") as f:
   writer = csv.writer(f)
   writer.writerow(results_train.keys())
   writer.writerows(zip(*results_train.values()))

iter:  100                 loss_tr:0.15106806933879852 acc_tr:0.781640625                 loss_te:0.14849595305785326 acc_te:0.7848703743706287
iter:  200                 loss_tr:0.15197409480810165 acc_tr:0.779765625                 loss_te:0.14837592084642867 acc_te:0.7852004800044315
iter:  300                 loss_tr:0.15094040282070637 acc_tr:0.779140625                 loss_te:0.14836082859358318 acc_te:0.784760339159361
iter:  400                 loss_tr:0.14954686537384987 acc_tr:0.783984375                 loss_te:0.14827234161571717 acc_te:0.7855305856382343
iter:  500                 loss_tr:0.15362040773034097 acc_tr:0.775078125                 loss_te:0.1487653419165544 acc_te:0.7828616928046858
iter:  600                 loss_tr:0.15232822209596633 acc_tr:0.77703125                 loss_te:0.14819563306133512 acc_te:0.7852004800044315
iter:  700                 loss_tr:0.153565194606781 acc_tr:0.773984375                 loss_te:0.1482018844132692 acc_te:0.785062397030037

### Finetune

In [ ]:
np.random.seed(114514)

class LinearLR():
    """Customized linear scheduler"""
    def __init__(self, optimizer, init_lr, total_iters=5):
        self.optimizer = optimizer
        self.lr_decay = init_lr / total_iters
    def step(self):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] -= self.lr_decay


# Parameters
lr = 6e-6
batch_size = 128 #64
maxiter = 4000
validation_per = 100
optimizer_finetune = optim.AdamW(bert_model.parameters(), lr=lr, weight_decay=3e-3)
scheduler = LinearLR(optimizer_finetune, init_lr=lr, total_iters=maxiter)

# Dataloader
train_loader = DataLoader(dataset_tr, batch_size=batch_size)#, num_workers=2)
valid_loader = DataLoader(dataset_te, batch_size=batch_size)#, num_workers=2)

# Start training
print("\n---------- Fine-tuning ----------\n")
results_finetune = build_model(bert_model, 
    lr = lr,
    train_loader = train_loader,
    test_loader = valid_loader,
    criterion = F.mse_loss,
    optimizer = optimizer_finetune,
    max_iter = maxiter,
    validate_per = validation_per,
    scheduler = scheduler
)



---------- Fine-tuning ----------

iter:  100                 loss_tr:0.13918346539139748 acc_tr:0.800390625                 loss_te:0.1300047397823401 acc_te:0.8128473666352285
iter:  200                 loss_tr:0.1326799276471138 acc_tr:0.80609375                 loss_te:0.12266028221224395 acc_te:0.8266859121725593
iter:  300                 loss_tr:0.12385779418051243 acc_tr:0.824375                 loss_te:0.11997642693385271 acc_te:0.8305651931695535
iter:  400                 loss_tr:0.11977500341832638 acc_tr:0.830625                 loss_te:0.11792772515139109 acc_te:0.8329859678174408
iter:  500                 loss_tr:0.12047812521457672 acc_tr:0.83140625                 loss_te:0.1155881705418439 acc_te:0.8392860226228204
iter:  600                 loss_tr:0.11687390655279159 acc_tr:0.835390625                 loss_te:0.11506907494974808 acc_te:0.8374693628767846
iter:  700                 loss_tr:0.11898512810468674 acc_tr:0.833515625                 loss_te:0.11576490880

In [ ]:
with open("../results/bert_tune_1.csv", "w") as f:
   writer = csv.writer(f)
   writer.writerow(results_train.keys())
   writer.writerows(zip(*results_finetune.values()))

## Save Model

In [ ]:
torch.save(bert_model, model_path)
print("\n---------- model saved ----------\n")


---------- model saved ----------



In [ ]:
# Save experiment results
# Combine training and fintuning results
results_all = {name:val+results_finetune[name] for name,val in results_train.items()}
train_maxiter = results_train["iter"][-1]
results_all["iter"] = results_train["iter"] + [i+train_maxiter for i in results_finetune["iter"]]

with open(result_path, "w") as f:
   writer = csv.writer(f)
   writer.writerow(results_all.keys())
   writer.writerows(zip(*results_all.values()))